In [ ]:
print("Installing dependencies from requirements.txt...")
!pip install -r requirements.txt
print("Dependencies installation complete.")


# This cell is designed to start the FastAPI application and expose it to the internet
# using an ngrok public tunnel. It integrates Uvicorn (the ASGI server) with ngrok.

print("Starting FastAPI application...") # Informative message indicating the start of the application.

import os # Module for interacting with the operating system (e.g., managing environment variables).
import threading # Module for running tasks in separate threads, allowing the notebook cell to remain active.
from IPython.display import display, HTML # For displaying HTML content (like clickable links) in the notebook output.
import time # For pausing execution (e.g., to allow services to start).
import subprocess  # For running external commands (like Uvicorn) as subprocesses.

# ====== IMPORTANT: This section ensures pyngrok is installed and ngrok executable is ready. ======
# It is duplicated here for robustness, although usually handled in a dedicated setup cell (Cell 12).
# This redundancy helps ensure `pyngrok` is available even if Cell 12 failed or was skipped.

print("Installing pyngrok...") # Inform the user that pyngrok is being installed.
!pip install pyngrok -q # Install the pyngrok Python library. `-q` ensures quiet output.
print("pyngrok installed.") # Confirmation message.

# Define paths for the ngrok zip file and the executable.
ngrok_zip_path = "ngrok-stable-linux-amd64.zip"
ngrok_executable_path = "ngrok"

# Check if the ngrok executable already exists to avoid re-downloading unnecessarily.
if not os.path.exists(ngrok_executable_path):
    print("Downloading ngrok executable...") # Inform the user about the download.
    # Download the ngrok executable zip file for Linux (Kaggle's environment).
    # `-O` specifies the output filename. `-q` is for quiet download progress.
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O {ngrok_zip_path} -q 
    !unzip -o {ngrok_zip_path} # Unzip the downloaded file. `-o` overwrites if exists.
    !chmod +x {ngrok_executable_path} # Grant execute permissions to the ngrok binary.
    print("Ngrok executable downloaded and configured.") # Confirmation message.
else:
    print("Ngrok executable already present.") # Inform if ngrok is already there.
# =================================================================================================

# Import ngrok only after ensuring its installation and executable are ready.
from pyngrok import ngrok # pyngrok is the Python wrapper for the ngrok service.

# Retrieve your ngrok authentication token. This token is essential for ngrok to operate.
# It should have been set as an environment variable in a previous setup cell (Cell 12).
ngrok_auth_token = "2xgYVbiMX6umq43cgdupTkDYOoA_6sEh5wdWfe7YBsTq7HhGf" # REPLACE this with your actual ngrok auth token.

# Validate the ngrok auth token. If it's not set, raise an error as ngrok won't function.
if ngrok_auth_token == "YOUR_NGROK_AUTH_TOKEN" or not ngrok_auth_token:
    print("🚨 Warning: Please replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok auth token from ngrok.com")
    print("Register for free at https://ngrok.com/signup and get your token from https://dashboard.ngrok.com/get-started/your-authtoken")
    raise ValueError("Ngrok auth token is not set. Cannot start tunnel.") 
else:
    os.environ["NGROK_AUTH_TOKEN"] = ngrok_auth_token # Set the token as an environment variable.
    ngrok.set_auth_token(ngrok_auth_token) # Authenticate pyngrok with the provided auth token.

# Define a function to run the Uvicorn server. This function will be executed in a separate thread.
def run_uvicorn():
    """
    Runs the Uvicorn server in a subprocess to host the FastAPI application.
    This function continuously monitors Uvicorn's output for startup confirmation.
    """
    try:
        # Command to start Uvicorn, serving the 'app' FastAPI instance from 'app.py' module
        # on all network interfaces (0.0.0.0) and port 8000.
        command = ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
        
        # Use subprocess.Popen to run Uvicorn. `Popen` is non-blocking, allowing the main thread to continue.
        # `stdout=subprocess.PIPE` and `stderr=subprocess.PIPE` capture Uvicorn's console output.
        # `text=True` decodes output as text.
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Read Uvicorn's output line by line to detect when the server has fully started.
        for line in process.stdout:
            print(f"Uvicorn: {line.strip()}") # Print Uvicorn's output to the notebook.
            if "Uvicorn running on" in line: # Look for a specific pattern indicating the server is active.
                print("Uvicorn server confirmed to be starting...") # Confirm Uvicorn startup.
                break # Exit the loop once the server is confirmed to be starting, to proceed with ngrok.
        
        # Wait for the Uvicorn process to complete. For a server, this means it will run indefinitely
        # unless it crashes or is explicitly stopped. This makes the thread (and thus the cell) remain active.
        process.wait()

    except FileNotFoundError:
        print("Error: 'uvicorn' command not found. Ensure uvicorn is installed (from requirements.txt).")
    except Exception as e:
        print(f"Error in Uvicorn thread: {e}")

try:
    # Create and start a new thread to run the `run_uvicorn` function.
    # This allows the FastAPI server to run in the background without blocking the notebook cell.
    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.daemon = True # Set the thread as a daemon; it will terminate when the main program exits.
    uvicorn_thread.start() # Start the Uvicorn thread.

    time.sleep(15) # Pause execution to allow time for Uvicorn to start and the FastAPI app (including model loading) to initialize.
    
    # Establish the ngrok tunnel. This exposes the locally running Uvicorn server (on port 8000)
    # to the internet, providing a public, accessible URL.
    public_url = ngrok.connect(8000).public_url
    print(f"Ngrok Tunnel URL: {public_url}") # Print the generated public URL.
    # Display a clickable HTML link to the FastAPI documentation (Swagger UI).
    display(HTML(f'<h2>Your FastAPI app is running at: <a href="{public_url}/docs" target="_blank">{public_url}/docs</a></h2>'))
    print("Uvicorn server started in a background thread.") # Confirmation message.

    uvicorn_thread.join() # Keep the notebook cell alive indefinitely while the Uvicorn server thread is active.

except Exception as e:
    # Catch any exceptions during ngrok or Uvicorn startup and log them.
    print(f"❌ Error starting ngrok or Uvicorn: {e}")
    print("Please check your ngrok auth token and ensure no other process is using port 8000.")